# C++



El programa crea un árbol de procesos, el proceso A crea tres procesos hijos, llamados B, C y D, el proceso hijo B crea otros dos procesos hijos, E y F, y el proceso D crea un último proceso hijo llamado G. Se le agregó tiempos de retraso para poder monitorear con el comando ps la correcta creación de los procesos.

In [82]:
%%writefile Fork.cpp
#include <iostream>
#include <unistd.h>
#include <sys/wait.h>

using namespace std;

void print_process_message(char char_of_process)
{
    cout << "Proceso " << char_of_process << " ( PID: " << getpid() << " | PPID: " << getppid() << " )" << endl;
}

int main(int argc, char *argv[])
{
    pid_t pidA, pidB, pidC, pidD, pidE, pidF, pidG;
    pidA = getpid();
    print_process_message('A');
    pidB = fork();
    if (pidB == 0)
    {
        pidB = getpid();
        print_process_message('B');
        pidE = fork();
        if (pidE == 0)
        {
            pidE = getpid();
            print_process_message('E');
            sleep(30);
            exit(0);
        }
        pidF = fork();
        if (pidF == 0)
        {
            pidF = getpid();
            print_process_message('F');
            sleep(30);
            exit(0);
        }
        sleep(30);
        wait(NULL);
        wait(NULL);
        exit(0);
    }
    pidC = fork();
    if (pidC == 0)
    {
        pidC = getpid();
        print_process_message('C');
        sleep(30);
        exit(0);
    }
    pidD = fork();
    if (pidD == 0)
    {
        pidD = getpid();
        print_process_message('D');
        pidG = fork();
        if (pidG == 0)
        {
            pidG = getpid();
            print_process_message('G');
            sleep(30);
            exit(0);
        }
        sleep(30);
        wait(NULL);
        exit(0);
    }
    sleep(30);
    wait(NULL);
    wait(NULL);
    wait(NULL);
    return 0;
}

Overwriting Fork.cpp


Verificamos que existe el archivo Fork.cpp

In [76]:
!ls -l

total 48
-rwxr-xr-x 1 root root 17744 Apr 12 12:56 Fork.bin
-rw-r--r-- 1 root root  1508 Apr 12 13:20 Fork.cpp
-rwx------ 1 root root  1401 Apr 12 13:11 main.py
-rw-r--r-- 1 root root   885 Apr 12 13:15 ProcessTree.java
-rw-r--r-- 1 root root   249 Apr 12 12:57 salidaC
-rw-r--r-- 1 root root   283 Apr 12 13:19 salidaJava
-rw-r--r-- 1 root root   318 Apr 12 13:11 salidaPython
drwxr-xr-x 1 root root  4096 Apr 10 13:31 sample_data


Compilamos el archivo Fork.cpp y generamos el binario.

In [77]:
%%shell
g++ Fork.cpp -o Fork.bin

Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaC"

In [78]:
%%shell
nohup ./Fork.bin 1> salidaC 2> /dev/null & 

Visualizamos los procesos generados por este programa

In [79]:
%%shell
ps -ef | grep Fork.bin

root       14688       1  0 13:21 ?        00:00:00 ./Fork.bin
root       14689   14688  0 13:21 ?        00:00:00 ./Fork.bin
root       14690   14688  0 13:21 ?        00:00:00 ./Fork.bin
root       14691   14688  0 13:21 ?        00:00:00 ./Fork.bin
root       14692   14689  0 13:21 ?        00:00:00 ./Fork.bin
root       14693   14689  0 13:21 ?        00:00:00 ./Fork.bin
root       14694   14691  0 13:21 ?        00:00:00 ./Fork.bin
root       14711     145  0 13:21 ?        00:00:00 /bin/bash -c ps -ef | grep Fork.bin 
root       14713   14711  0 13:21 ?        00:00:00 grep Fork.bin


Ahora visualizamos el arbol de procesos, ejecutando el siguiente comando:
***pstree -pc PID***

**PID**: PID del proceso A (Lo pueden encontrar dentro del archivo "salidaC")

In [80]:
%%shell
pstree -pc 14688

Fork.bin(14688)─┬─Fork.bin(14689)─┬─Fork.bin(14692)
                │                 └─Fork.bin(14693)
                ├─Fork.bin(14690)
                └─Fork.bin(14691)───Fork.bin(14694)


Visualizamos la salida de este programa

In [81]:
!cat salidaC

Proceso A (PID: 14688 | PPID: 1 )
Proceso B (PID: 14689 | PPID: 14688 )
Proceso D (PID: 14691 | PPID: 14688 )
Proceso F (PID: 14693 | PPID: 14689 )
Proceso E (PID: 14692 | PPID: 14689 )
Proceso C (PID: 14690 | PPID: 14688 )
Proceso G (PID: 14694 | PPID: 14691 )



# Java




El programa recibe 2 argumentos, la cantidad de hijos que debe generar el proceso al ejecutarse, y el nivel de jerarquia que este tiene. Para cumplimentar el ejercicio tiene un corte cuando alcanzan el nivel 2.

In [55]:
%%writefile ProcessTree.java
package processTree;

import java.io.IOException;

public class ProcessTree {
	
	public static void main(String[] args) throws IOException, InterruptedException {
		
		ProcessHandle process = ProcessHandle.current();
		long pid = process.pid();
		long ppid = process.parent().get().pid();
		System.out.println("Soy el proceso: "+pid+", Mi papa es: "+ppid);
		
		int level = Integer.parseInt(args[1]);
		if(level==2) {
			Thread.sleep(10000);
			return;
		}	
		
		int childrenNum = Integer.parseInt(args[0]);
		
		Process childs[] = new Process[childrenNum];
		
		for (int i = 1; i <= childrenNum; i++) {
			ProcessBuilder pb = new ProcessBuilder("java","ProcessTree.java",String.valueOf(childrenNum-i),String.valueOf(level+1));
			pb.inheritIO();
			childs[i-1] = pb.start();
		}
		Thread.sleep(10000);
		for (int i = 0; i < childs.length; i++) {
			childs[i].waitFor();
		}
		
	}
	
}

Writing ProcessTree.java


Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaJava"

In [70]:
!nohup java ProcessTree.java 3 0 1>salidaJava 2>/dev/null &

Visualizamos los procesos que produce este programa.

In [71]:
!ps -ef | grep ProcessTree.java

root       14067       1 61 13:19 ?        00:00:01 java ProcessTree.java 3 0
root       14090   14067 52 13:19 ?        00:00:01 java ProcessTree.java 2 1
root       14094   14067 53 13:19 ?        00:00:01 java ProcessTree.java 1 1
root       14098   14067 40 13:19 ?        00:00:00 java ProcessTree.java 0 1
root       14148     145  0 13:19 ?        00:00:00 /bin/bash -c ps -ef | grep ProcessTree.java
root       14154   14148  0 13:19 ?        00:00:00 grep ProcessTree.java


Vemos el arbol de procesos que genero este programa.


In [72]:
!pstree -pT 14067

java(14067)─┬─java(14090)─┬─java(14164)
            │             └─java(14166)
            ├─java(14094)───java(14191)
            └─java(14098)


Mostramos la salida del programa

In [73]:
!cat salidaJava

Soy el proceso: 14067, Mi papa es: 1
Soy el proceso: 14090, Mi papa es: 14067
Soy el proceso: 14094, Mi papa es: 14067
Soy el proceso: 14098, Mi papa es: 14067
Soy el proceso: 14164, Mi papa es: 14090
Soy el proceso: 14166, Mi papa es: 14090
Soy el proceso: 14191, Mi papa es: 14094



# Python



El programa crea un árbol de procesos, el proceso A crea tres procesos hijos, llamados B, C y D, el proceso hijo B crea otros dos procesos hijos, E y F, y el proceso D crea un último proceso hijo llamado G. Se le agregó tiempos de retraso para poder monitorear con el comando ps la correcta creación de los procesos.

In [50]:
%%writefile main.py
import os
import sys
import time

def print_process_message(char_of_process):
    print("Soy el proceso " + char_of_process + " ( PID: " + str(os.getpid()) + " | PPID: " + str(os.getppid()) + " )")
    sys.stdout.flush()
	
def parent():

    print_process_message('A')
    pidB = os.fork()

    if pidB == 0:
        pidB = os.getpid()
        print_process_message('B')
        pidE = os.fork()
        if pidE == 0:
            pidE = os.getpid()
            print_process_message('E')
            time.sleep(30)
            
            os._exit(0)
        
        pidF = os.fork()
        if pidF == 0:
            pidF = os.getpid()
            print_process_message('F')
            time.sleep(30)
            os._exit(0)
        
        time.sleep(30)
        os.wait()
        os.wait()
        os._exit(0)
    
    pidC = os.fork()
    if pidC == 0:
        pidC = os.getpid()
        print_process_message('C')
        time.sleep(30)
        os._exit(0)
    
    pidD = os.fork()

    if pidD == 0:
        pidD = os.getpid()
        print_process_message('D')
        pidG = os.fork()
        if pidG == 0:
            pidG = os.getpid()
            print_process_message('G')
            time.sleep(30)
            os._exit(0)
        
        time.sleep(30)
        os.wait()
        os._exit(0)
    
    os.wait()
    os.wait()
    os.wait()
    time.sleep(30)

    return 0

parent()

Overwriting main.py


Cambiamos los permisos para ejecutar.

In [ ]:
!chmod 700 ./main.py

Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaPython"

In [51]:
!nohup python main.py 1>salidaPython 2>/dev/null &

Visualizamos los procesos en ejecucion del programa "main.py"



In [52]:
!ps -ef | grep main.py

root       11583       1  1 13:11 ?        00:00:00 python3 main.py
root       11585   11583  0 13:11 ?        00:00:00 python3 main.py
root       11586   11583  0 13:11 ?        00:00:00 python3 main.py
root       11587   11583  0 13:11 ?        00:00:00 python3 main.py
root       11588   11585  0 13:11 ?        00:00:00 python3 main.py
root       11589   11585  0 13:11 ?        00:00:00 python3 main.py
root       11590   11587  0 13:11 ?        00:00:00 python3 main.py
root       11599     145  0 13:11 ?        00:00:00 /bin/bash -c ps -ef | grep main.py
root       11601   11599  0 13:11 ?        00:00:00 /bin/bash -c ps -ef | grep main.py


Visualizamos el arbol de procesos que genera este programa.

In [53]:
!pstree -pc 11583

python3(11583)─┬─python3(11585)─┬─python3(11588)
               │                └─python3(11589)
               ├─python3(11586)
               └─python3(11587)───python3(11590)


Visualizamos la salida por consola

In [54]:
!cat salidaPython

Soy el proceso A ( PID: 11583 | PPID: 1 )
Soy el proceso B ( PID: 11585 | PPID: 11583 )
Soy el proceso C ( PID: 11586 | PPID: 11583 )
Soy el proceso D ( PID: 11587 | PPID: 11583 )
Soy el proceso E ( PID: 11588 | PPID: 11585 )
Soy el proceso F ( PID: 11589 | PPID: 11585 )
Soy el proceso G ( PID: 11590 | PPID: 11587 )
